In [1]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
#!pip install transformers
#! git clone https://github.com/jmunozmendi/SNGP

In [ ]:
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer
import numpy as np

In [ ]:
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import MultiStepLR
import torch.nn.functional as F
from torch import nn
import torch
import os
import torchvision.datasets as dset
import torchvision.transforms as transforms
#import gpytorch
import math
import tqdm

# Load dataset

In [ ]:
import pandas as pd

dataset = pd.read_csv('dataset.tsv', sep='\t')

dataset['text'] = dataset['text'].astype(str)

dataset.head()

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples,
                           padding='max_length',  # Pad to max_length
                           truncation=True,       # Truncate to max_length
                           max_length=200)        # Specify the max length


In [ ]:
dataset['tokenized_text'] = dataset['text'].apply(lambda x: preprocess_function(x))

In [ ]:
input_ids = dataset['tokenized_text'].apply(lambda x: x['input_ids'])
attention_masks = dataset['tokenized_text'].apply(lambda x: x['attention_mask'])

In [ ]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


input_ids_tensor = torch.tensor(input_ids)
attention_masks_tensor = torch.tensor([item for item in attention_masks])
labels_tensor = torch.tensor(dataset['label'].values)

# Custom Dataset class
class DistilBERTDataset(Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        # Return a tuple where the first element is a tuple of input_ids and attention_mask,
        # and the second element is labels
        return (self.input_ids[idx], self.attention_masks[idx]), self.labels[idx]


dataset = DistilBERTDataset(input_ids_tensor, attention_masks_tensor, labels_tensor)

# Split dataset
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = train_test_split(dataset, train_size=train_size, test_size=test_size, stratify=labels_tensor)


## Creating the DistilBERT Model

In [ ]:
class DistillBERTClass(torch.nn.Module):
    def __init__(self, num_outputs = 768):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, num_outputs)


    def forward(self, X):
        (input_ids, attention_mask) = X

        output_1 = self.l1(input_ids=input_ids.long(), attention_mask=attention_mask.long())
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        output = self.pre_classifier(pooler)

        return output


num_inputs_gp = 768
feature_extractor = DistillBERTClass(num_outputs = num_inputs_gp)

## Creating the GP Layer

In the next cell, we create the layer of Gaussian process models that are called after the neural network. In this case, we'll be using one GP per feature, as in the SV-DKL paper. The outputs of these Gaussian processes will the be mixed in the softmax likelihood.

In [ ]:
from SNGP.sngp import SNGP

## Creating the full SVDKL Model

With both the DenseNet feature extractor and GP layer defined, we can put them together in a single module that simply calls one and then the other, much like building any Sequential neural network in PyTorch. This completes defining our DKL model.

In [ ]:
model = SNGP(out_features=2, backbone=feature_extractor, backbone_output_features = num_inputs_gp, num_inducing=1024, momentum = 0)


In [ ]:
print(model)

## Defining Training and Testing Code

Next, we define the basic optimization loop and testing code. This code is entirely analogous to the standard PyTorch training loop. We create a `torch.optim.SGD` optimizer with the parameters of the neural network on which we apply the standard amount of weight decay suggested from the paper, the parameters of the Gaussian process (from which we omit weight decay, as L2 regualrization on top of variational inference is not necessary), and the mixing parameters of the Softmax likelihood.

We use the standard learning rate schedule from the paper, where we decrease the learning rate by a factor of ten 50% of the way through training, and again at 75% of the way through training.

In [ ]:
model.train_model(dataset=train_dataset, epochs=25, batch_size=32, weight_decay=0.01, lr = 2e-5)

In [ ]:
info = model.predict(dataset=test_dataset, batch_size=256)

In [ ]:
test_x, test_y = zip(*test_dataset)

In [ ]:
test_x, test_y = np.array(test_x), np.array(test_y)

In [ ]:
result = np.logical_and(info.decision, test_y)

In [ ]:
print("Accuracy: ", result.sum() / len(result))

In [ ]:
info